In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/content/info.csv', index_col = 0)
df.drop(['data_path'], axis=1, inplace=True)

# 1. 전처리

In [6]:
def text_preprocessor(s):
    import re
    
    ## (1) [], (), {}, <> 괄호와 괄호 안 문자 제거하기
    pattern = r'\([^)]*\)' 
    s = re.sub(pattern=pattern, repl='', string=s)
    
    pattern = r'\[[^)]*\]' 
    s = re.sub(pattern=pattern, repl='', string=s)
    
    pattern = r'\<[^)]*\>' 
    s = re.sub(pattern=pattern, repl='', string=s)
    
    pattern = r'\{[^)]*\}'
    s = re.sub(pattern=pattern, repl='', string=s)
    
    ## (2) '...' 제거
    s = s.replace('...', ' ')
    s = s.replace('...', ' ')
    
    ## (3) 특수문자 제거
    pattern = r'[^a-zA-Z가-힣]'
    s = re.sub(pattern=pattern, repl=' ', string=s)
    
    ## (4) 단위 제거: cm, km, etc.
    units = ['mm', 'cm', 'km', 'ml', 'kg', 'g']
    for unit in units:
        s = s.lower() # 대문자를 소문자로 변환
        s = s.replace(unit, '')
        
    # (5) 공백 기준으로 분할하기
    s_split = s.split()
    
    # (6) 글자 1개만 있으면 제외하기
    s_list = []
    for word in s_split:
        if len(word) !=1:
            s_list.append(word)
            
    return s_list


In [8]:
from konlpy.tag import Okt  
from konlpy.tag import Kkma    # NLP of the Korean language

def words_tokonizer(text):
   
    kkma = Kkma()
    
    words = []
    
    # Text preprocessing using the UDF above
    s_list = text_preprocessor(text)
    
    # POS tagging
    for s in s_list:
        words_ = kkma.pos(s)   
        
        # NNG indexing
        for word in words_:
            if word[1] == 'NNG':
                words.append(word[0])
            
    return words

In [9]:
df['tokenized'] = df['story'].apply(lambda text: words_tokonizer(text))

In [10]:
# 단어별 빈도수, 단어길이가 2개 이상인 단어만 추출

vocab = {}
preprocessed_sentences = []

# 단어 토큰화 진행한 tokenized 변수 이용
for tokenized_sentence in df['tokenized']:
    result = []

    for word in tokenized_sentence:
        
        if len(word) >= 2:          # 단어길이가 2개 이상인 단어만 추출
            result.append(word)
            
            if word not in vocab:   # 딕셔너리 {key=단어, value=빈도수}
                vocab[word] = 0
            vocab[word] += 1

    preprocessed_sentences.append(result)

df['preprocessed_list'] = preprocessed_sentences

In [11]:
df["preprocessed"] = ""

for i in range(len(df['preprocessed_list'])):
    df['preprocessed'][i] = ' '.join(s for s in df['preprocessed_list'][i])
df.head()

,genre,title,story,tokenized,preprocessed_list,preprocessed
0,일상,피터팬날다,한국콘텐츠진흥원과 함께하는 2009 네이버 웹툰 공모전 가작 수상작\n\n소년의 마...,"[한국, 콘텐츠, 진흥원, 웹, 공모전, 가작, 수상작, 소년, 마음, 피, 태풍,...","[한국, 콘텐츠, 진흥원, 공모전, 가작, 수상작, 소년, 마음, 태풍, 현실, 적...",한국 콘텐츠 진흥원 공모전 가작 수상작 소년 마음 태풍 현실 적응 애착 느림 철학
1,일상,가우스전자,다국적 문어발 기업 가우스전자 마케팅3부 이야기\n\n곽백수 작가가 선보이는 본격 ...,"[다국적, 문어발, 기업, 가우스, 전자, 마케팅, 이야기, 곽, 백수, 작가, 본...","[다국적, 문어발, 기업, 가우스, 전자, 마케팅, 이야기, 백수, 작가, 본격, ...",다국적 문어발 기업 가우스 전자 마케팅 이야기 백수 작가 본격 직장인 만화
2,일상,미쳐 날뛰는 생활툰,만화가가 되고 싶었던 나날\n\n가볍게 그려본 생활툰 하나에 격동하는 주변의 이야기,"[만화가, 나날, 생활, 하나, 격동, 주변, 이야기]","[만화가, 나날, 생활, 하나, 격동, 주변, 이야기]",만화가 나날 생활 하나 격동 주변 이야기
3,일상,생활의참견,"쉽게 지나치는 일상과 지나간 추억 속에서 발견하는 유쾌한 웃음,\n\n삶의 유머를 ...","[일상, 추억, 속, 발견, 웃음, 삶, 유머, 황, 당, 유, 추억, 생활, 만화]","[일상, 추억, 발견, 웃음, 유머, 추억, 생활, 만화]",일상 추억 발견 웃음 유머 추억 생활 만화
4,일상,닥터앤닥터 육아일기,"산부인과 의사 엄마의 임신과 출산, 공학박사 아빠의 논문 기반 육아.\n\n완벽할 ...","[산부인과, 의사, 엄마, 임신, 출산, 공학, 박사, 아빠, 논문, 기반, 육아,...","[산부인과, 의사, 엄마, 임신, 출산, 공학, 박사, 아빠, 논문, 기반, 육아,...",산부인과 의사 엄마 임신 출산 공학 박사 아빠 논문 기반 육아 완벽 앞길 대로 사람...


## 전처리 끝

# TF-IDF

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
df['preprocessed'].isnull().sum()

0

In [14]:
# 전처리 끝난 열 df['preprocessed']에 대해서 TF-IDF 행렬 구하고 크기 출력
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['preprocessed'])
print('TF-IDF 행렬의 크기(shape) :', tfidf_matrix.shape)

# 2024행을 가진 7490의 열
# 즉, 2024개의 웹툰을 표현하기 위해 7490개의 단어가 사용됐다는 것(7490차원의 문서가 2024개 있는것이다.)
# 7490차원 임베딩

TF-IDF 행렬의 크기(shape) : (2024, 7490)


In [15]:
# 코사인 유사도 측정
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :',cosine_sim.shape)

# 2024개의 각 문서(웹툰) 벡터와 자기 자신을 포함한 2024개의 문서간의 유사도가 기록된 행렬이다.

코사인 유사도 연산 결과 : (2024, 2024)


In [27]:
# 여기서 2024개 데이터에 대한 인덱스와 title 딕션너리를 만들어 둔다.
title_to_index = dict(zip(df['title'], df.index))
# 웹툰 제목을 치면 해당 행에 대한 인덱스가 나오는 구조

In [17]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 웹툰의 타이틀로부터 해당 웹툰의 인덱스를 받아온다.
    idx = title_to_index[title]

    # 해당 웹툰과 모든 웹툰과의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬한다.
    sim_webtoon = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 웹툰의 인덱스를 얻는다.
    movie_indexes = [idx[0] for idx in sim_webtoon[1:]]
    score_indexes = [idx[1] for idx in sim_webtoon[1:]]

    # 가장 유사한 10개의 웹툰의 제목을 리턴한다.
    title = df['title'].iloc[movie_indexes].to_frame().reset_index()
    genre = df['genre'].iloc[movie_indexes].to_frame()
    score = pd.DataFrame(score_indexes)

    wt_df = pd.concat([title, genre, score], axis=1).iloc[:10, :]
    wt_df.columns = ['index', 'title', 'genre', 'score']
    wt_df['index'] = wt_df['index'].astype(int)

    return wt_df

In [18]:
get_recommendations('김부장')

,index,title,genre,score
0,1510,장난을 잘 치는 전 타카기 양,일상,0.200485
1,524,내 최애가 위험해,일상,0.181941
2,693,"뜬금없지만, 오늘 만나러 와 줘",일상,0.151832
3,1161,"아저씨와, 사랑 [개정판]",일상,0.151083
4,1930,쌈빡,일상,0.130387
5,1847,MZ-레이징 인페르노,일상,0.129849
6,1941,에이전트,일상,0.125259
7,1300,옆집남자 친구,일상,0.123788
8,431,껄끄러운 상사와 이웃이 되었습니다,일상,0.122566
9,106,도토리 문화센터,일상,0.119527


# FastText

In [19]:
from gensim.models.fasttext import FastText
from gensim.test.utils import datapath

In [20]:
corpus = []
for words in df['preprocessed']:
    corpus.append(words.split())

In [21]:
fasttext_model = FastText(size=200, 
                          window=2, 
                          min_count=2, 
                          workers=4, 
                          sg=0)
fasttext_model.build_vocab(corpus)
fasttext_model.train(corpus, total_examples=fasttext_model.corpus_count, epochs=10) 

## 단어 벡터의 평균 구하기

In [22]:
def get_document_vectors(document_list):
    document_embedding_list = []

    # 각 문서에 대해서
    for line in document_list:
        doc2vec = None
        count = 0
        for word in line.split():
            if word in fasttext_model.wv.vocab:
                count += 1
                # 해당 문서에 있는 모든 단어들의 벡터값을 더한다.
                if doc2vec is None:
                    doc2vec = fasttext_model[word]
                else:
                    doc2vec = doc2vec + fasttext_model[word]

        if doc2vec is not None:
            # 단어 벡터를 모두 더한 벡터의 값을 문서 길이로 나눠준다.
            doc2vec = doc2vec / count
            document_embedding_list.append(doc2vec)

    # 각 문서에 대한 문서 벡터 리스트를 리턴
    return document_embedding_list

In [23]:
document_embedding_list = get_document_vectors(df['preprocessed'])
print('문서 벡터의 수 :',len(document_embedding_list))

문서 벡터의 수 : 2019


<ipython-input-22-2036d01c0bc0>:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  doc2vec = fasttext_model[word]
<ipython-input-22-2036d01c0bc0>:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  doc2vec = doc2vec + fasttext_model[word]


In [24]:
# 각 벡터간 코사인 유사도

cosine_similarities = cosine_similarity(document_embedding_list, document_embedding_list)
print('코사인 유사도 매트릭스의 크기 :', cosine_similarities.shape)

코사인 유사도 매트릭스의 크기 : (2019, 2019)


## 추천 시스템 구현

In [25]:
def fasttext_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 웹툰의 타이틀로부터 해당 웹툰의 인덱스를 받아온다.
    idx = title_to_index[title]

    # 해당 웹툰와 모든 웹툰과의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_similarities[idx]))

    # 유사도에 따라 웹툰 정렬
    sim_webtoon = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 웹툰의 인덱스를 얻는다.
    movie_indexes = [idx[0] for idx in sim_webtoon[1:]]
    score_indexes = [idx[1] for idx in sim_webtoon[1:]]

    # 가장 유사한 10개의 웹툰의 제목을 리턴한다.
    title = df['title'].iloc[movie_indexes].to_frame().reset_index()
    genre = df['genre'].iloc[movie_indexes].to_frame()
    score = pd.DataFrame(score_indexes)

    wt_df = pd.concat([title, genre, score], axis=1).iloc[:10, :]
    wt_df.columns = ['index', 'title', 'genre', 'score']
    wt_df['index'] = wt_df['index'].astype(int)

    return wt_df

In [26]:
fasttext_recommendations("김부장")

,index,title,genre,score
0,304,거만한 부사장에게 잡혔습니다,일상,0.999989
1,456,나랑 살래？,일상,0.999987
2,1288,열애 수업,일상,0.999986
3,888,블루밍 시퀀스 [GL],일상,0.999986
4,474,나의 스마트보이,일상,0.999986
5,813,반짝반짝 [독점 연재],일상,0.999985
6,1163,아주 특별한 연애,일상,0.999985
7,1152,아사쿠라씨의 익애에 휩쓸리지 않을 거예요!,일상,0.999985
8,1224,어차피 남편은!,일상,0.999984
9,1048,속삭이듯 사랑을 노래하다 [GL],일상,0.999984
